In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

import re
import tensorflow as tf
import transformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/Transformers/kaggle/data.csv'
data = pd.read_csv(path)

In [5]:
data.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [6]:
data.shape

(5842, 2)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [8]:
data['Sentiment'].value_counts(dropna=False)

neutral     3130
positive    1852
negative     860
Name: Sentiment, dtype: int64

In [9]:
# import string

In [10]:
# string.punctuation

In [11]:
# def preprocess_data():
#     pass

In [12]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [13]:
model_checkpoint = 'yiyanghkust/finbert-tone'

In [14]:
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

In [15]:
def tokenize_data(data, text):
    tokenizer_ouput= tokenizer(data[text].tolist(), padding=True)
    return dict(tokenizer_ouput)

In [16]:
tokenized_data = tokenize_data(data, 'Sentence')

In [17]:
tokenized_df = pd.DataFrame(tokenized_data)

In [18]:
merged = pd.concat([data, tokenized_df], axis=1)

In [19]:
merged.head()

,Sentence,Sentiment,input_ids,token_type_ids,attention_mask
0,The GeoSolutions technology will leverage Bene...,positive,"[3, 6, 25902, 17873, 8769, 341, 36, 763, 15843...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,"[3, 30805, 4767, 823, 19, 13787, 585, 269, 308...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"For the last quarter of 2010 , Componenta 's n...",positive,"[3, 14, 6, 168, 47, 7, 453, 585, 1234, 363, 56...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,According to the Finnish-Russian Chamber of Co...,neutral,"[3, 1782, 9, 6, 3388, 19072, 85, 8667, 23116, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,The Swedish buyout firm has sold its remaining...,neutral,"[3, 6, 23063, 11984, 803, 49, 449, 38, 761, 21...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [20]:
label = merged['Sentiment'].map({'negative':1, 'neutral':0, 'positive':2})

In [21]:
label = np.array(label)

In [22]:
label

array([2, 1, 2, ..., 0, 0, 2])

In [23]:
tokzzz = np.array(list(merged['input_ids']))

In [24]:
tf_data = tf.data.Dataset.from_tensor_slices((tokzzz, label)).batch(64)

In [25]:
tf_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 134), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [26]:
model = TFBertForSequenceClassification.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [27]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=tf.optimizers.Adam(5e-6), loss=loss, metrics=['accuracy'])

In [28]:
# model.compile(optimizer=tf.optimizers.Adam())

In [29]:
model.fit(tf_data, epochs=10)

Epoch 1/10
92/92 [==============================] - 212s 2s/step - loss: 1.0608 - accuracy: 0.5433
Epoch 2/10
92/92 [==============================] - 151s 2s/step - loss: 0.8712 - accuracy: 0.6027
Epoch 3/10
92/92 [==============================] - 150s 2s/step - loss: 0.7821 - accuracy: 0.6450
Epoch 4/10
92/92 [==============================] - 151s 2s/step - loss: 0.6864 - accuracy: 0.6962
Epoch 5/10
92/92 [==============================] - 151s 2s/step - loss: 0.5943 - accuracy: 0.7419
Epoch 6/10
92/92 [==============================] - 150s 2s/step - loss: 0.5074 - accuracy: 0.7725
Epoch 7/10
92/92 [==============================] - 150s 2s/step - loss: 0.4517 - accuracy: 0.8013
Epoch 8/10
92/92 [==============================] - 150s 2s/step - loss: 0.4120 - accuracy: 0.8198
Epoch 9/10
92/92 [==============================] - 150s 2s/step - loss: 0.3859 - accuracy: 0.8230
Epoch 10/10
92/92 [==============================] - 150s 2s/step - loss: 0.3485 - accuracy: 0.8401


In [30]:
# model.predict(sample_data)

In [55]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer,return_all_scores=False)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [56]:
pipe(data['Sentence'][707])

[{'label': 'Negative', 'score': 0.9999285936355591}]

In [46]:
data['Sentiment'][707]

'negative'

In [47]:
data['Sentence'][707]

'The net sales of the whole fiscal year 2008 will be lower than in 2007 and operating profit is estimated to be negative .'